In [1]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf

2022-08-17 14:33:48.873026: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-17 14:33:48.873068: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from sklearn.datasets import make_moons

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

X = np.array([[1,1],[1.5,1.5], [2.5,2.5], [2,2], [3.5,3.5], [3,3],[4,4], [5,5], [6,6],[7,7],[8,8],[9,9]])
X = X / 9
y = X**2

In [3]:
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)
layers = 10

@qml.qnode(dev)
def qnode(inputs, weights):
    #print(inputs.shape)
    #qml.AngleEmbedding(inputs, wires=range(n_qubits))
    for i, j in enumerate(inputs):
        qml.RX(j, wires=i)
    used = 0
    for i in range(layers):
        for j in range(n_qubits):
            qml.RY(weights[used + j], wires = j)
        used = used + n_qubits
    #qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [4]:
n_layers = 6
weight_shapes = {"weights": (layers*n_qubits,)}

In [5]:
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)

2022-08-17 14:33:50.965110: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-17 14:33:50.965155: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-17 14:33:50.965179: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tomskopfbahnhof): /proc/driver/nvidia/version does not exist
2022-08-17 14:33:50.965482: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
inputs = tf.keras.layers.Input(shape=(2,))
clayer_1 = tf.keras.layers.Dense(2)
clayer_2 = tf.keras.layers.Dense(2)
bigd = tf.keras.layers.Dense(10)
bigg = tf.keras.layers.Dense(10)
model = tf.keras.models.Sequential([clayer_1, qlayer, bigd, bigg, clayer_2])

In [7]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(opt, loss="mse", metrics=["mse"])

In [8]:
fitting = model.fit(X, y, epochs=100, batch_size=2)


Epoch 1/100
6/6 [==============================] - 1s 215ms/step - loss: 0.5302 - mse: 0.5302
Epoch 2/100
6/6 [==============================] - 1s 183ms/step - loss: 0.4747 - mse: 0.4747
Epoch 3/100
6/6 [==============================] - 1s 189ms/step - loss: 0.4276 - mse: 0.4276
Epoch 4/100
6/6 [==============================] - 1s 193ms/step - loss: 0.3874 - mse: 0.3874
Epoch 5/100
6/6 [==============================] - 1s 186ms/step - loss: 0.3527 - mse: 0.3527
Epoch 6/100
6/6 [==============================] - 1s 167ms/step - loss: 0.3232 - mse: 0.3232
Epoch 7/100
6/6 [==============================] - 1s 180ms/step - loss: 0.2975 - mse: 0.2975
Epoch 8/100
6/6 [==============================] - 1s 170ms/step - loss: 0.2748 - mse: 0.2748
Epoch 9/100
6/6 [==============================] - 1s 180ms/step - loss: 0.2553 - mse: 0.2553
Epoch 10/100
6/6 [==============================] - 1s 175ms/step - loss: 0.2383 - mse: 0.2383
Epoch 11/100
6/6 [==============================] - 1s 191m

In [11]:
model.predict([[0.5,0.5]])

array([[0.30589446, 0.3043422 ]], dtype=float32)

In [12]:
model.predict([[0.1,0.1]])

array([[0.30589446, 0.3043422 ]], dtype=float32)